## Goes over modeling, starting from modeling tables.
### We're using modeling tables which were prepared based on 12 hours worth of vital sign data from each patient, as well as medication history during the stay, and patient characteristics.
### The model predicts the probability of having a rapid response team event in 1 hour's time from the time of prediction. A RRT event is called after personnel identify that a patient has an urgent need for medical service.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
# import datetime as datetime
import cPickle as pickle
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier #, RandomForestClassifier, 
from sklearn.ensemble.partial_dependence import plot_partial_dependence, partial_dependence
from sklearn.grid_search import GridSearchCV

### function definitions

In [ ]:
def score_printout(X_test, y_test, fittedModel):
    print "AUC-ROC Score of model: ", roc_auc_score(y_test, fittedModel.predict_proba(X_test)[:,1])
    print "Precision Score of model: ", precision_score(y_test, fittedModel.predict(X_test))
    print "Recall Score of model: ", recall_score(y_test, fittedModel.predict(X_test))

In [ ]:
def make_feature_importance_plot(featuresAndImportances, numFeatures):
    topN = featuresAndImportances[:numFeatures]
    labels = [pair[0] for pair in topN]
    values = [pair[1] for pair in topN]
    ind = np.arange(len(values)+2)
    width = 0.35   
    plt.barh(range(numFeatures),values)
    ax = plt.subplot(111)
    ax.set_yticks(ind+width)
    ax.set_yticklabels(labels, rotation=0, size=12)
    plt.ylabel('Feature', size=20)
    plt.xlabel('Importance', size=20)
    plt.show()


### Read in data

We did not share our modeling data, so you will have to create your own. The pipeline tool can help you do this. If you save the results to a csv, `masterdf_rrt` and `masterdf_nonrrt` are dataframes with the modeling data for each of the positive and negative classes, respectively. 

In [ ]:
masterdf_rrt = pd.read_csv('RRT_modeling_table_13hr_raw.csv')
masterdf_nonrrt = pd.read_csv('NonRRT_modeling_table_13hr_raw.csv')

### Look at summary statistics for numeric columns for rrt & non-rrt tables (35 cols)

In [ ]:
masterdf_rrt.columns

In [ ]:
masterdf_rrt.describe().T

In [ ]:
masterdf_nonrrt.describe().T

### We have a good amount of nan values in some columns. Lets plot the nan values to get a sense of how many there are

In [ ]:
def show_df_nans(masterdf, collist=None):
    '''
    Create a data frame for features which may be nan.
    Make nan values be 1, numeric values be 0
    A heat map where dark squares/lines show where data is missing.
    '''
    if not collist:
        plot_cols = ['obese','DBP_mean', 'DBP_recent', 'SBP_mean', 'SBP_recent', 'HR_mean', 'HR_recent',
               'MAP_mean', 'MAP_recent', 'temp_mean', 'temp_recent', 'SPO2_mean',
               'SPO2_recent', 'RR_mean', 'RR_recent', 'pulse_mean', 'pulse_recent',
               'CO2_mean', 'CO2_recent', 'GCS_mean', 'GCS_recent']
    else:
        plot_cols = collist 
    
    df_viznan = pd.DataFrame(data = 1,index=masterdf.index,columns=plot_cols)
    df_viznan[~pd.isnull(masterdf[plot_cols])] = 0
    plt.figure(figsize=(10,8))
    plt.title('Dark values are nans')
    return sns.heatmap(df_viznan.astype(float))

In [ ]:
# subset of numeric columns we'll use in modeling (sufficient data available)
plot_cols_good = ['obese','DBP_mean', 'DBP_recent', 'SBP_mean', 'SBP_recent', 
               'MAP_mean', 'MAP_recent', 'temp_mean', 'temp_recent', 'SPO2_mean',
               'SPO2_recent', 'RR_mean', 'RR_recent', 'pulse_mean', 'pulse_recent']

In [ ]:
show_df_nans(masterdf_nonrrt)  # show all columns that may have nans
# show_df_nans(masterdf_nonrrt, plot_cols_good)  # show the columns whch we plan to use for modeling

In [ ]:
show_df_nans(masterdf_rrt)
# show_df_nans(masterdf_rrt, plot_cols_good)

### Let's not use those columns where there are significant nans: drop HR (heart rate; we have pulse rate instead), CO2, and GCS, which leaves us with 28 features.

In [ ]:
col_use = ['age', 'sex', 'obese', 'smoker', 'prev_rrt', 'on_iv', 'bu-nal', 'DBP_mean',
       'DBP_recent', 'SBP_mean', 'SBP_recent',
       'MAP_mean', 'MAP_recent', 'temp_mean', 'temp_recent', 'SPO2_mean',
       'SPO2_recent', 'RR_mean', 'RR_recent', 'pulse_mean', 'pulse_recent',
       'anticoagulants', 'narcotics', 'narc-ans', 'antipsychotics',
       'chemo', 'dialysis', 'race']

In [ ]:
X_rrt = masterdf_rrt[col_use]
X_notrrt = masterdf_nonrrt[col_use]

### We need to deal with these nans before we can start modeling. (There should not be any nans in the modeling table)

In [ ]:
# let's look at getting rid of the data rows where vitals signs are all nans
vitals_cols = ['DBP_mean', 'DBP_recent', # take the mean of all the measurements & the most recently observed point
            'SBP_mean', 'SBP_recent',
            'MAP_mean', 'MAP_recent', # mean arterial pressure
             'temp_mean', 'temp_recent',# temperature
             'SPO2_mean', 'SPO2_recent',
            'RR_mean', 'RR_recent', # respiratory rate
            'pulse_mean', 'pulse_recent']

In [ ]:
# Write out rows that are not all 0/NaNs across. (if all nans, remove this sample)
X_rrt = X_rrt.loc[np.where(X_rrt.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]]
X_rrt = X_rrt.reset_index(drop=True)
X_notrrt = X_notrrt.loc[np.where(X_notrrt.ix[:, vitals_cols].sum(axis=1, skipna=True)!=0)[0]]
X_notrrt = X_notrrt.reset_index(drop=True)

In [ ]:
# if 'obese' is Nan, then set the patient to be not obese.
X_rrt.loc[np.where(pd.isnull(X_rrt['obese']))[0], 'obese'] = 0
X_notrrt.loc[np.where(pd.isnull(X_notrrt['obese']))[0], 'obese'] = 0

### Let's see how X_rrt & X_notrrt look

In [ ]:
show_df_nans(X_rrt, vitals_cols)

In [ ]:
show_df_nans(X_notrrt, vitals_cols)

### Some columns have significant missing values.

In [ ]:
print X_rrt[['pulse_mean', 'pulse_recent']].describe().T
print "size of X_rrt: "+str(len(X_rrt))
print
print X_notrrt[['pulse_mean', 'pulse_recent']].describe().T
print "size of X_notrrt: " + str(len(X_notrrt))

### We have plenty of samples for the non-RRT case. We can delete off rows with values that are missing without concern that we'll lose negtive examples for RRT events for modeling.

In [ ]:
# DROP THE ROWS WHERE PULSE IS NAN
X_notrrt = X_notrrt.ix[np.where(pd.isnull(X_notrrt['pulse_mean'])!=True)[0]]
X_notrrt = X_notrrt.reset_index(drop=True)
# And similarly for all rows with significant nans:
X_notrrt = X_notrrt.ix[np.where(pd.isnull(X_notrrt['RR_mean'])!=True)[0]]
X_notrrt = X_notrrt.reset_index(drop=True)
X_notrrt = X_notrrt.ix[np.where(pd.isnull(X_notrrt['MAP_mean'])!=True)[0]]
X_notrrt = X_notrrt.reset_index(drop=True)
X_notrrt = X_notrrt.ix[np.where(pd.isnull(X_notrrt['temp_mean'])!=True)[0]]
X_notrrt = X_notrrt.reset_index(drop=True)
X_notrrt = X_notrrt.ix[np.where(pd.isnull(X_notrrt['SPO2_mean'])!=True)[0]]
X_notrrt = X_notrrt.reset_index(drop=True)


In [ ]:
all_cols = ['age', 'sex', 'obese', 'smoker', 'prev_rrt', 'on_iv', 'bu-nal',
       'DBP_mean', 'DBP_recent', 'SBP_mean', 'SBP_recent', 'MAP_mean',
       'MAP_recent', 'temp_mean', 'temp_recent', 'SPO2_mean',
       'SPO2_recent', 'RR_mean', 'RR_recent', 'pulse_mean', 'pulse_recent',
       'anticoagulants', 'narcotics', 'narc-ans', 'antipsychotics',
       'chemo', 'dialysis', 'race']

In [ ]:
show_df_nans(X_notrrt, all_cols)

### Still need to deal with nans in X_rrt. Temp & pulse are the most of concern

In [ ]:
X_rrt[['temp_mean', 'pulse_mean']].describe().T

### We'll impute missing values in X_rrt after combining that data with X_notrrt, and use the mean from each column after merging to fill the values.

In [ ]:
# add labels to indicate positive or negative class
X_rrt['label'] = 1
X_notrrt['label'] = 0

# Combine the tables
XY = pd.concat([X_rrt, X_notrrt])
XY = XY.reset_index(drop=True)
y = XY.pop('label')
X = XY

# Fill nans with mean of columns
X = X.fillna(X.mean())

In [ ]:
# map genders to 1/0
X['is_male'] = X['sex'].map({'M': 1, 'F': 0})
X.pop('sex')

In [ ]:
X.race.value_counts()

In [ ]:
# we won't use race in modeling
X.pop('race')

In [ ]:
show_df_nans(X, vitals_cols)

In [ ]:
X.columns

In [ ]:
X.describe().T

# Modeling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
print len(y_train)
print len(y_train[y_train]==1)

In [ ]:
len(y_test[y_test==1])

In [ ]:
Xscaled = StandardScaler().fit_transform(X)
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xscaled, y, test_size=0.3)

## Gradient Boosting Classifier - Unscaled (with partial dependence plots below)


In [ ]:
paramGrid = {'n_estimators': [100, 200, 300],
             'learning_rate': [0.1, 0.05, 0.01, 0.2],
             'max_depth': [3, 4, 5, 6],
             'min_samples_leaf': [1, 2],
             'subsample': [0.75, 1.0, 0.85],
             'loss': ['deviance'],
             'max_features': [None, 'auto']
            }

gs = GridSearchCV(GradientBoostingClassifier(), 
                  param_grid=paramGrid, 
                  scoring='roc_auc', 
                  n_jobs=-1, 
                  cv=5, 
                  verbose=10)

gs.fit(X_train, y_train)

# Result:
# GradientBoostingClassifier(init=None, learning_rate=0.05, loss='deviance',
#               max_depth=3, max_features=None, max_leaf_nodes=None,
#               min_samples_leaf=2, min_samples_split=2,
#               min_weight_fraction_leaf=0.0, n_estimators=300,
#               presort='auto', random_state=None, subsample=0.75, verbose=0,
#               warm_start=False)

## Grid search for best GBC - Scaled (with partial dependece plots below)

In [ ]:
paramGrid = {'n_estimators': [100, 200, 300],
             'learning_rate': [0.1, 0.05, 0.01, 0.2],
             'max_depth': [3, 4, 5, 6],
             'min_samples_leaf': [1, 2],
             'subsample': [0.75, 1.0, 0.85],
             'loss': ['deviance'],
             'max_features': [None, 'auto']
            }

gss = GridSearchCV(GradientBoostingClassifier(), 
                  param_grid=paramGrid, 
                  scoring='roc_auc', 
                  n_jobs=-1, 
                  cv=5, 
                  verbose=10)

gss.fit(Xs_train, ys_train)

# Result:
# GradientBoostingClassifier(init=None, learning_rate=0.05, loss='deviance',
#               max_depth=3, max_features='auto', max_leaf_nodes=None,
#               min_samples_leaf=1, min_samples_split=2,
#               min_weight_fraction_leaf=0.0, n_estimators=300,
#               presort='auto', random_state=None, subsample=0.75, verbose=0,
#               warm_start=False)


## How different are best estimators for scaled & unscaled data?

In [ ]:
gbc = GradientBoostingClassifier(init=None, learning_rate=0.05, loss='deviance',
               max_depth=3, max_features=None, max_leaf_nodes=None,
               min_samples_leaf=2, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=300,
               presort='auto', random_state=None, subsample=0.75, verbose=0,
               warm_start=False)
gbc.fit(X_train, y_train)
score_printout(X_test, y_test, gbc)
print classification_report(y_test, gbc.predict(X_test))
confusion_matrix(y_test, gbc.predict(X_test))

In [ ]:
# gbcs = gss.best_estimator_
# gbcs.fit(Xs_train, ys_train)
# score_printout(Xs_test, ys_test, gbc)
# print classification_report(ys_test, gbcs.predict(Xs_test))
# confusion_matrix(ys_test, gbcs.predict(Xs_test))

### Use unscaled data -- better results & easier interpretability

In [ ]:
# Let's plot the confusion matrix so it's a little clearer
plt.figure()
sns.set(font_scale=1.5)
sns.heatmap(confusion_matrix(y_test, gbc.predict(X_test)), annot=True, fmt='d')

## Let's look at the most important features in this model

In [ ]:
gbcRankedFeatures = sorted(zip(X.columns, gbc.feature_importances_), 
                          key=lambda pair: pair[1], 
                          reverse=False)

In [ ]:
plt.figure()
make_feature_importance_plot(gbcRankedFeatures, 27) # note - we have 27 features currently


### Let's look a partial dependence plots
#### If the partial dependence is high, then the model for that given value of that given feature is more likely to predict an rrt result.
#### Will not show more complex interactions -- if importance is high but partial dependence is marginal, this may be due to interactions

In [ ]:
fig, axs = plot_partial_dependence(gbc, X_train, range(0, 6, 1), feature_names=X.columns.get_values(), n_jobs=-1, grid_resolution=50)
plt.subplots_adjust(top=0.9)

In [ ]:
fig, axs = plot_partial_dependence(gbc, X_train, range(6, 12, 1), feature_names=X.columns.get_values(), n_jobs=-1, grid_resolution=50)
plt.subplots_adjust(top=0.9)

In [ ]:
fig, axs = plot_partial_dependence(gbc, X_train, range(12, 18, 1), feature_names=X.columns.get_values(), n_jobs=-1, grid_resolution=50)
plt.subplots_adjust(top=0.9)

In [ ]:
fig, axs = plot_partial_dependence(gbc, X_train, range(18, 24, 1), feature_names=X.columns.get_values(), n_jobs=-1, grid_resolution=50)
plt.subplots_adjust(top=0.9)

In [ ]:
fig, axs = plot_partial_dependence(gbc, X_train, range(24, 27, 1), feature_names=X.columns.get_values(), n_jobs=-1, grid_resolution=50)
plt.subplots_adjust(top=0.9)

## Use 3-D plot to investigate feature interactions for weak partial dependence plots... (weak effect may be masked by stronger interaction with other features)

In [ ]:
names = X_train.columns
zip(range(len(names)), names)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# not all features may work for this viz
fig = plt.figure(figsize=(10,8))
target_feature = (16, 18)  # <--  change the two numbers here to determine what to plot up
pdp, (x_axis, y_axis) = partial_dependence(gbc, target_feature, X=X_train, grid_resolution=50)
XX, YY = np.meshgrid(x_axis, y_axis)
Z = pdp.T.reshape(XX.shape).T
ax = Axes3D(fig)
surf = ax.plot_surface(XX, YY, Z, rstride=1, cstride=1, cmap=plt.cm.BuPu)
ax.set_xlabel(names[target_feature[0]])
ax.set_ylabel(names[target_feature[1]])
ax.set_zlabel('Partial dependence')
#  pretty init view
ax.view_init(elev=22, azim=122)
plt.colorbar(surf)
plt.suptitle('')
plt.subplots_adjust(top=0.9)

plt.show()

## From Model to Risk Score


In [ ]:
# Return probabilities from the model, rather than predictions
y_proba = gbc.predict_proba(X_test)

In [ ]:
# note - y_proba contains probabilities for class 0 in column 0 & probabilities for class 1 in column 1.
# we're only interested in the probability for class 1
y_proba

In [ ]:
pred_probs = pd.DataFrame(data=y_proba[:,1], columns =["model_probability_of_rrt"], index = X_test.index)

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
pred_probs['model_probability_of_rrt'] = pd.to_numeric(pred_probs.model_probability_of_rrt)

In [ ]:
pred_probs.hist(bins = 20, xlabelsize = 16, ylabelsize=16)
plt.tick_params(labelsize=14)
plt.title("Model output probabilities")
plt.ylabel('Count', fontsize=14)

### We see that although we see more values close to 0 and 1, we also see that the model outputs a full range of probabilities, which would translate well into risk scores.


### Patient Risk Score = model probability * 10
The score should be rounded to whole values to give the sense that this is not an exact measure.

In [ ]:
pred_probs['score'] = pred_probs['model_probability_of_rrt'].apply(lambda x: int(round(x*10.0, 0)))

In [ ]:
pred_probs.head()

In [ ]:
pred_probs.score.value_counts()

### Save model

In [ ]:
from sklearn.externals import joblib
# joblib.dump(gbc, 'gbc_base.pkl') # note - if left uncompressed, this writes a whole lot of supporting numpy files.
joblib.dump(gbc, 'my_trained_model.compressed', compress=True)  

# to unpack: joblib.load(filename)

### Save modeling table

In [ ]:
# Create combined data frame including modeling table, rrt label, and proability associated with result
df = pd.concat([X_test, pred_probs, y_test],axis=1, join_axes=[X_test.index])

In [ ]:
df.head()

In [ ]:
# May need to rename columns to get rid of dash in name...
df.rename(columns={'bu-nal': 'bu_nal', 'narc-ans': 'narc_ans'}, inplace=True)
df.to_csv('ModelingTable_with_results.csv')